In [ ]:
import re

import Bio.Restriction as Restriction
import pandas as pd
import pygsheets
import requests
import toml
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from cytoolz import partial
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -U --no-deps primer3plus

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.geneious as geneious
import paulssonlab.cloning.commands.parser as cmd_parser
import paulssonlab.cloning.commands.semantics as cmd_semantics
import paulssonlab.cloning.design as design
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.workflow as workflow

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

# Config

In [ ]:
olib_oligos = reg[("oLIB", "oligos")]
olt_oligos = reg[("oLT", "oligos")]
plib_plasmids = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]
flib_fragments = reg[("fLIB", "fragments")]
part_types = reg[("fLIB", "fragments", "Part types")]

# Ingest missing parts

## Double Terminators

In [ ]:
def ingest_part(id_, base_row, storage_vector_id, storage_enzyme):
    name = flib_fragments[id_]["Name"]
    # this correctly prefers the annealed version of Voigt_DT100 (gBlock was bad)
    oligo_row = olt_oligos.find({"Name": f"{name}_sense"})
    if oligo_row is None:
        oligo_row = olt_oligos.find({"Name": name})
    if oligo_row is None:
        raise ValueError(f"cannot find oligo for '{name}'")
    plasmid_id = plib_plasmids.find_id({"Names": name})
    if plasmid_id is None:
        raise ValueError(f"cannot find plasmid for '{name}'")
    usage = f"{plasmid_id}/{part_enzyme}"
    if oligo_row["Name"].endswith("_sense"):
        storage_usage = "{}={}".format(
            oligo_row["ID"],
            olt_oligos.find_id({"Name": f"{name}_antisense"}),
        )
    else:
        storage_usage = oligo_row["ID"]
    usage += f",{storage_usage}/{part_enzyme}"
    seq = workflow.re_digest_part(oligo_row["Sequence"], part_enzyme)
    flib_fragments[id_] = {
        **base_row,
        "Name": name,
        "Usage": usage,
        "Sequence": seq.seq_lower(),
    }
    command = (
        f"@GG({storage_usage}/{storage_enzyme}, {storage_vector_id}/{storage_enzyme})"
    )
    plib_plasmids.upsert(
        {"ID": plasmid_id, "Description": base_row["Description"], "Command": command}
    )
    plasmid_seq = reg.eval_command(command)["_seq"]
    plib_maps[plasmid_id] = plasmid_seq

In [ ]:
double_terminators = [
    id_ for id_, row in flib_fragments.items() if row["Name"].startswith("Voigt_DT")
]

In [ ]:
part_type = "Terminator_DE"
overhangs = workflow.overhangs_for(part_types[part_type])
part_enzyme = Restriction.BsaI
storage_enzyme = Restriction.BsmBI
storage_vector_id = "pLIB112"

base_row = {
    "Upstream overhang": overhangs[0],
    "Downstream overhang": overhangs[1],
    "Type": part_type,
    "Author": "Jacob Quinn Shenker",
    "Date": workflow.date(),
    "Species/codon usage": "E. coli",
    "Description": "Voigt strong double terminator",
    "Reference": "Park, Y., Espah Borujeni, A., Gorochowski, T.E., Shin, J. and Voigt, C.A., 2020. P recision design of stable genetic circuits carried in highly‐insulated E. coli genomic landing pads. Molecular systems biology, 16(8), p.e9584.",
}

In [ ]:
for id_ in double_terminators:
    ingest_part(id_, base_row, storage_vector_id, storage_enzyme)

## RiboJs

In [ ]:
part_type = "5UTR_1"
overhangs = workflow.overhangs_for(part_types[part_type])
part_enzyme = Restriction.BsaI
storage_enzyme = Restriction.BsmBI
storage_vector_id = "pLIB112"

base_row = {
    "Upstream overhang": overhangs[0],
    "Downstream overhang": overhangs[1],
    "Type": part_type,
    "Author": "Jacob Quinn Shenker",
    "Date": workflow.date(),
    "Species/codon usage": "E. coli",
    "Description": "Voigt ribozyme insulator",
    "Reference": "Nielsen, A.A., Der, B.S., Shin, J., Vaidyanathan, P., Paralanov, V., Strychalski, E.A., Ross, D., Densmore, D. and Voigt, C.A., 2016. Genetic circuit design automation. Science, 352(6281), p.aac7341.",
}

In [ ]:
for id_ in [f"fLIB{n}" for n in range(325, 341)]:
    ingest_part(id_, base_row, storage_vector_id, storage_enzyme)

In [ ]:
flib_fragments.commit()

In [ ]:
plib_plasmids.commit()

In [ ]:
plib_maps.commit()

# NAO strains

In [ ]:
nao_sheets = gc.open_by_key("1nuEeeEwU4Rj4HFjwCWKi6P9sapH2-mPDf4XSDQBiptk")
tus_sheet = nao_sheets.worksheet(value=0)
strains_sheet = nao_sheets.worksheet(value=2)
tus = tus_sheet.get_as_df().set_index("tLIB number")
strains = strains_sheet.get_as_df()

In [ ]:
flib_fragments_df = reg[("fLIB", "fragments")].remote
plib_plasmids_df = reg[("pLIB", "plasmids")].remote
lib_strains_df = reg[("LIB", "strains")].remote

In [ ]:
plasmid_wells = lib_strains_df[lib_strains_df["Names"].str.len() != 0]
plasmid_wells = plasmid_wells[plasmid_wells["Reference"].str.startswith("Halleran")]
plasmid_wells = plasmid_wells[["Plasmids", "Other Notes"]]
plasmid_wells["Well"] = plasmid_wells["Other Notes"].map(
    lambda s: next(iter(re.findall(r"Well: ([A-Z0-9]+)", s)), None)
)
plasmid_wells = plasmid_wells[~plasmid_wells["Well"].isnull()]

In [ ]:
well_to_plasmid = dict(plasmid_wells[["Plasmids", "Well"]].values[:, ::-1])

In [ ]:
name_case_corrections = ["PsigW", "sigW", "rsiW"]
name_corrections = {
    "SCFP3A": "SCFP3A_nocut",
    "mKate2": "mKate2_nocut",
    "mVenus": "mVenus_nocut",
}
for dt_num in (5, 54, 100, 101, 104, 3, 56, 60):
    name_corrections[f"DT{dt_num}"] = f"Voigt_DT{dt_num}"
name_corrections = {
    **{k.lower(): k for k in name_case_corrections},
    **{k.lower(): v for k, v in name_corrections.items()},
}

In [ ]:
# @cache
def _resolve_nao_part(reg, parts_sheet, well_to_plasmid, name_corrections, name):
    # LIB number: LIB262
    if name.startswith("LIB"):
        strain = reg.get(name)
        plasmids = re.split(r"\s*,\s*", strain["Plasmids"])
        if len(plasmids) != 1:
            raise ValueError(
                f"expecting a single plasmid for strain {name}: {plasmids}"
            )
        plasmid = plasmids[0]
        return _resolve_nao_part(
            reg, parts_sheet, well_to_plasmid, name_corrections, plasmid
        )
    elif name.startswith("pLIB"):
        parts_using_plasmid = parts_sheet["Usage"].str.contains(f"{name}/")
        if (num_parts := parts_using_plasmid.sum()) != 1:
            raise ValueError(
                f"expecting a single part that uses plasmid {name}, found {num_parts}"
            )
        part_name = parts_sheet[parts_using_plasmid].iloc[0]["Name"]
        return part_name
    # pLIB number: 199
    elif name.isdigit():  # isdigit isn't robust, but fine for this
        return _resolve_nao_part(
            reg, parts_sheet, well_to_plasmid, name_corrections, f"pLIB{name}"
        )
    # well number: A11
    elif re.match(r"(1|2)?[A-H][0-9]{1,2}b?", name):
        well = name
        if well[-1] == "b":
            well = f"2{well[:-1]}"
        if well[0] not in ("1", "2"):
            well = f"1{well}"
        plasmid = well_to_plasmid[well]
        return _resolve_nao_part(
            reg, parts_sheet, well_to_plasmid, name_corrections, plasmid
        )
    # part capitalization: Psigw -> PsigW, SigW -> sigW
    elif name.lower() in name_corrections:
        return name_corrections[name.lower()]
    # part
    return name


# @cache
def _parse_nao_part(reg, parts_sheet, well_to_plasmid, name_corrections, name):
    name = str(name)  # numbers may be parsed as ints
    names = [re.sub(r"\s*\([^)]*\)\s*", "", n) for n in re.split(r"\s*\+\s*", name)]
    return [
        _resolve_nao_part(reg, parts_sheet, well_to_plasmid, name_corrections, n)
        for n in names
    ]


parse_nao_part = partial(
    _parse_nao_part, reg, flib_fragments_df, well_to_plasmid, name_corrections
)


def nao_tu_to_command(row):
    uns_a = str(row["UNS_A"]).replace("*", "_r")
    uns_e = str(row["UNS_E"]).replace("*", "_r")
    promoter = parse_nao_part(row["Promoter (pLIB number or 3G part ID) "])
    rbs = parse_nao_part(row["Ribosome binding site (pLIB number or 3G part ID) "])
    cds = parse_nao_part(row["Coding region (pLIB number or 3G part ID) "])
    term = parse_nao_part(row["Terminator (pLIB number or 3G part ID) "])
    parts = [f"UNS{uns_a}_A", *promoter, *rbs, *cds, *term, f"UNS{uns_e}_E"]
    command = f"@GG({', '.join(parts)})"
    return command


def nao_strain_to_command(row, tus):
    command = "@Gib({}, {})".format(
        ", ".join(
            [
                nao_tu_to_command(tus.loc[f"tLIB{t}"])
                for t in re.split(r",|-", str(row["TUs"]))
            ]
        ),
        backbone_pcrs[row["Plasmids"]],
    )

In [ ]:
strains.tail()

In [ ]:
nao_strain_to_command(strains.loc[716], tus)

In [ ]:
test_names = [
    "199",
    "DT3",
    "Psigw",
    "A4b",
    "LIB254",
    "ScmJ+fLIB238",
    "mVenus",
    "SCFP3A",
    "fLIB311",
    "358 (pTet)",
    "331 (LacI) +199+SCFP3A",
    "ScmJ+fLIB314",
    "323 (PhlF)+199+mVenus",
    "RiboJ+oLT81",
    "RiboJ+PH_BsaI",
    "sigw+Deg_PH",
    "RsiW+Deg_PH",
    "Barcode PH",
    "Long Barcode PH",
]
for name in test_names:
    print(
        name,
        "->",
        parse_nao_part(name),
    )

In [ ]:
reg.get("Deg_PH")

In [ ]:
reg.get("oLT81")

In [ ]:
reg.get("fLIB311")

In [ ]:
nao_tu_to_command(tus.loc["tLIB447"])

In [ ]:
nao_part_names = pd.melt(tus.iloc[:, 3:7])["value"].value_counts()
with pd.option_context("display.max_rows", None):
    print(nao_part_names)

In [ ]:
nao_part_failures = set()
for nao_part_name in tqdm(nao_part_names.index[:]):
    try:
        part_names = parse_nao_part(nao_part_name)
        print(nao_part_name, "->", part_names)
        for part_name in part_names:
            res = reg.get(part_name)
            if not len(res["_seq"]):
                raise
    except:
        # raise
        nao_part_failures.add((nao_part_name, part_name))

In [ ]:
nao_part_failures

In [ ]:
for nao_part_name in list(nao_part_failures)[:10]:
    part_names = parse_nao_part(nao_part_name)
    print(nao_part_name, "->", part_names)
    for part_name in part_names:
        res = reg.get(part_name)
        if not len(res["_seq"]):
            raise

In [ ]:
strains["Plasmids"].unique()

In [ ]:
backbone_pcrs = {
    "": "V37m_UNS1_UNS10",
    "p15A": "JUMP_p15a_UNS1_UNS10",
    "pSC101": "JUMP_pSC101_UNS1_UNS10",
}